In [1]:
## NEED TO RUN THIS TO GET CHOLMOD TO WORK
import ctypes

CUDA_LIB = "/share/software/user/open/cuda/12.2.0/targets/x86_64-linux/lib"
OPENBLAS = "/share/software/user/open/openblas/0.3.20/lib"
SUITESPARSE = "/share/software/user/open/suitesparse/7.4.0/lib64"

libs_to_load = [
    f"{OPENBLAS}/libopenblas.so.0",
    f"{CUDA_LIB}/libnvrtc.so.12",
    f"{CUDA_LIB}/libcublas.so.12",
    f"{CUDA_LIB}/libcublasLt.so.12",
    f"{CUDA_LIB}/libnvrtc-builtins.so.12.2",
    f"{SUITESPARSE}/libcholmod.so.5",
]

# Load each library with global visibility
for lib in libs_to_load:
    print(f"Loading {lib} ...")
    ctypes.CDLL(lib, mode=ctypes.RTLD_GLOBAL)

# Import sksparse safely
import sksparse.cholmod as cholmod
print("✓ sksparse imported successfully")


Loading /share/software/user/open/openblas/0.3.20/lib/libopenblas.so.0 ...
Loading /share/software/user/open/cuda/12.2.0/targets/x86_64-linux/lib/libnvrtc.so.12 ...
Loading /share/software/user/open/cuda/12.2.0/targets/x86_64-linux/lib/libcublas.so.12 ...
Loading /share/software/user/open/cuda/12.2.0/targets/x86_64-linux/lib/libcublasLt.so.12 ...
Loading /share/software/user/open/cuda/12.2.0/targets/x86_64-linux/lib/libnvrtc-builtins.so.12.2 ...
Loading /share/software/user/open/suitesparse/7.4.0/lib64/libcholmod.so.5 ...
✓ sksparse imported successfully


In [2]:
import cvxpy as cp
import numpy as np
import time
import clarabel
import torch
from zap.admm import ADMMSolver, WeightedADMMSolver
from zap.conic.cone_bridge import ConeBridge
import scipy.sparse as sp
from experiments.conic_solve.benchmarks.max_flow_benchmark import MaxFlowBenchmarkSet

from zap.conic.cone_utils import get_standard_conic_problem



In [3]:
## Create a large problem that is valid using max flow benchmark set 
n = 10000
base_seed = 42
max_flow_benchmark = MaxFlowBenchmarkSet(num_problems=1, n=n, base_seed=base_seed)

for prob in max_flow_benchmark:
    problem = prob
num_variables = sum(np.prod(var.shape) for var in problem.variables())
num_constraints = sum(constraint.size for constraint in problem.constraints)
nnz = num_variables # This is the actual number of edges
total_possible_edges = float(n*(n - 1))
density = nnz/total_possible_edges
sparsity = 1 - density
print(f'Generated a valid network with {n} nodes using starting seed {base_seed}')
print(f"Actual Number of Edges: {nnz}")
print(f"Total Possible Edges: {total_possible_edges}")
print(f"Graph sparsity: {sparsity}")
print(f"Graph density: {density}")
print(f"Number of Variables: {num_variables}")
print(f"Number of Constraints: {num_constraints}")

Generated a valid network with 10000 nodes using starting seed 42
Actual Number of Edges: 183993
Total Possible Edges: 99990000.0
Graph sparsity: 0.9981598859885988
Graph density: 0.0018401140114011401
Number of Variables: 183993
Number of Constraints: 193993


In [4]:
# Get conic problem form so we can (i) solve standard conic form and (ii) solve using ZAP
cone_params, data, cones = get_standard_conic_problem(problem, solver=cp.CLARABEL)

In [6]:
# Solve the conic form using ZAP
machine = 'cuda'
dtype = torch.float32
cone_bridge = ConeBridge(cone_params)
cone_admm_devices = [d.torchify(machine=machine, dtype=dtype) for d in cone_bridge.devices]
cone_admm = ADMMSolver(
    machine=machine,
    dtype=dtype,
    atol=1e-6,
    rtol=1e-6,
    alpha=1,
    adaptive_rho=False,
    tau=2,
    num_iterations=10000,
)
start_time = time.time()
cone_solution_admm, cone_history_admm = cone_admm.solve(
    net=cone_bridge.net, devices=cone_admm_devices, time_horizon=cone_bridge.time_horizon
)
end_time = time.time()
solve_time = end_time - start_time
obj_val = cone_solution_admm.objective
print(f"Objective value: {obj_val}")
print(f"Time taken: {solve_time:.4f} seconds")



ADMM converged in 6449 iterations.
Objective value: -158.00318908691406
Time taken: 30.9725 seconds


In [21]:
## Solve using CVXPY and CLARABEL from Zap formulation
start_time = time.time()
outcome = cone_bridge.solve(solver=cp.CLARABEL)
end_time = time.time()
solve_time = end_time - start_time
obj_val = outcome.problem.value
problem_status = outcome.problem.status
num_iters = outcome.problem.solver_stats.num_iters
print(f"Objective value: {obj_val}")
print(f"Time taken: {solve_time:.4f} seconds")
print(f"Number of iterations: {num_iters}")
print(f"Status: {problem_status}")



Objective value: -157.99999999370155
Time taken: 987.8816 seconds
Number of iterations: 12
Status: optimal
